In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [3]:
import re
import string
from nltk.tokenize import TweetTokenizer

#This step is done in Sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens

#remove punctuations -- this method is not in use
# def rm_punctuation(tweet):
#     RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
#     print(f'this is the punctuation  {RE_PUNCTUATION}')
#     print("\n")
#     newValue = tweet.replace(RE_PUNCTUATION, "")
#     return newValue



In [4]:
#This step is done in Sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [5]:
from nltk.corpus import stopwords

# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

[' said', ' also', '#climatecrisis', 'too', ' are', "i've", ' about', ' did', 'weren', 'no', " could've", 'themselves', 'mightn', 'theirs', ' must', " where'll", "wouldn't", ' like', ' what', ' yet', ' us', 'now', 'off', 'yourselves', ' a', 'i', '#GreenNewDeal', " must've", " wouldn't", 'all', 'any', ' her', ' into', 'but', ' will', ' only', 'myself', ' own', ' cannot', 'with', "he'd", "she'd", 'my', ' my', ' does', ' an', ' could', " he'd", ' neither', " shouldn't", " she's", 'as', " what's", 'needn', ' let', '#bushfires', ' can', "we're", ' am', ' on', ' other', 'each', " who'll", 'whom', 'he', ' we', 'down', 'only', "shan't", ' them', ' than', 'same', ' dear', " he'll", ' may', 'where', ' of', "couldn't", 'y', ' able', 'm', 'll', ' some', 'until', ' but', " we'll", 'should', 'this', 'their', "doesn't", ' at', ' i', 'during', " there's", 'own', 'haven', ' all', 'for', ' ever', "how's", ' either', "isn't", 'am', 'against', "won't", 'very', 'if', " why'll", ' whom', 'so', ' this', 're'

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [8]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# text_Vect = TfidfVectorizer()
# X_Vect  = text_Vect.fit_transform(X)
from sklearn.feature_extraction.text import TfidfVectorizer
text_Vect = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
X_Vect = text_Vect.fit_transform(X)

c:\users\tasne\envs\newproject\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [10]:
X_train.shape

(50683, 87818)

In [11]:
y_train.shape

(50683,)

In [12]:
X_test.shape

(21722, 87818)

In [13]:
y_test.shape

(21722,)

In [14]:
lr = LR(solver='sag')

In [15]:
lr.fit(X_train, y_train)  

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
predictions = lr.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
print(confusion_matrix(y_test,predictions))

[[ 414    1   18   41   26   34    6    3    1   16    0    8]
 [   0  227    0   33    6    0    2    0    2    0    0    0]
 [   8    0 1256  217   59  255   19   23   24   26    3   23]
 [  26    5  212 3933  117  257   58   13  124   50   13   49]
 [  13    8   68  176 1015  121   14    8   25   39    2    6]
 [   6    4   66  121   30 5170   18   48   22   14    1    7]
 [   4    0   13   84    9   11 1185    0   25    3    7   70]
 [   2    4   32   30    9  329    5  486    4    4    3    3]
 [   0    0    5   83   20   34   17    2 1077    6    1    1]
 [  10    0   11   63   27   45    2    3    5 1200    4    7]
 [   0    0    3   12    5    9   23    1   12    0  349   16]
 [   3    0    9   47    7   11   48    0   21    3    5 1583]]


In [19]:
print(classification_report(y_test,predictions))

                   precision    recall  f1-score   support

    #actonclimate       0.85      0.73      0.79       568
       #bushfires       0.91      0.84      0.87       270
   #climateaction       0.74      0.66      0.70      1913
   #climatechange       0.81      0.81      0.81      4857
   #climatecrisis       0.76      0.68      0.72      1495
   #climatestrike       0.82      0.94      0.88      5507
     #environment       0.85      0.84      0.84      1411
#fridaysforfuture       0.83      0.53      0.65       911
   #globalwarming       0.80      0.86      0.83      1246
    #greennewdeal       0.88      0.87      0.88      1377
   #savetheplanet       0.90      0.81      0.85       430
  #sustainability       0.89      0.91      0.90      1737

         accuracy                           0.82     21722
        macro avg       0.84      0.79      0.81     21722
     weighted avg       0.82      0.82      0.82     21722



In [20]:
from sklearn import metrics

In [21]:
metrics.accuracy_score(y_test,predictions)

0.8238191695055704

Accuracy is --- 
- Accuracy is -- 0.8241414234416721 -- without tokens and stopwords
- Accuracy is -- 0.8239112420587423 -- with tokenization and stopwords
> Note adding tokens and stopwords does effect  accuracy a little bit

In [22]:
strVal =  ["Cuyahoga county eliminates use of plastic bags","Weather is still warm in winters and is not freezing"]
Value = text_Vect.transform(strVal)
Value.shape

(2, 87818)

In [23]:
lr.predict(Value)

array(['#environment', '#climatechange'], dtype=object)

In [24]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np

scores = cross_val_score(LogisticRegression(solver='saga'), X_train, y_train)
print("Mean cross-validation accuracy: {:.2f}".format(np.mean(scores)))

Mean cross-validation accuracy: 0.83


In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1]}
grid = GridSearchCV(LogisticRegression(solver='sag'), param_grid)
grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

Best cross-validation score: 0.83
Best parameters:  {'C': 1}



- Best cross-validation score: 0.83
- Best parameters:  {'C': 1}

In [26]:
grid.predict(Value)

array(['#environment', '#climatechange'], dtype=object)